# 分治算法
我们如果把一个大问题改变成两个长度减半的小问题(问题没变)，再将小问题继续拆分，直到拆分成长度为1的问题返回并组合。其实这分治法自上而下将问题进行拆解，而动态规划是自下而上把问题解决(通常是比较难推导)，分治法我们在04二叉树的递归套路里已经有体现了(把问题下放到子树上，这就是一种分治)。分治法三步骤：
- 分解：将问题分成两部分，更小长度的子问题
- 解决：实现一个递归函数，输入问题，输出结果
- 合并：对两个部分分别求解之后，合并结果

## 问题1：[为运算表达式设计优先级](https://leetcode-cn.com/problems/different-ways-to-add-parentheses/)
### 题目
给定一个只包含加、减和乘法的数学表达式，求通过加括号可以得到多少种不同的结果。例如：表达式为"2-1-1",可以得到结果((2-1)-1) = 0 和(2-(1-1)) = 2，所以我们输出[0,2].再比如"2\*3-4\*5",可以有以下组合(2*(3-(4\*5))) = -34 ，((2\*3)-(4*5)) = -14 ...
### 分析
- 所有的加、减、乘，都能看成x op y的形式，而x，y仍可以继续写成x op y的形式，直到x，y是数字为止。
- 因此我们的递归函数返回条件就是当输入的仅仅是数字(isdigit())则返回该数字，否则进入递归形式.
- 要注意递归函数的返回应该是一个数组[ ],里面包括有当前表达式可以组成的所有结果。例如"2\*3-4\*5"，分解成"2"和"3-4*5",其中第一部分返回[2],第二部分返回表达式为"3-4\*5"的所有可能结果[-17,-5],因此2需要和[-17，-5]中每一个子结果相乘，因此该种分解的结果是[-34,-10]。
### 优化
- 我们可以采用一个字典来储存当前表达式的可以有的结果，每次进入递归形式时先判定是否已经计算过了

In [5]:
cash = dict()
def process(expression:str):
    if expression.isdigit():
        return [int(expression)]
    # 判断该表达式是否已经计算过了
    if expression in cash:
        return cash[expression]

    res = []
    for i, chr in enumerate(expression):
        if chr in ['+','*','-']:
            left_res = process(expression[:i]) # 返回该子问题的结果[2]
            right_res = process(expression[i+1:]) # 返回该子问题的结果[-34，-10]
            # 合并两个子问题的结果
            for left in left_res:
                for right in right_res:
                    if chr == '+':
                        res.append(left+right)
                    elif chr == '*':
                        res.append(left*right)
                    else:
                        res.append(left-right)
    cash[expression] = res
    return res
process("2*3-4*5")

[-34, -10, -14, -10, 10]

## 问题2：[漂亮数组](https://leetcode-cn.com/problems/beautiful-array/)
### 题目
对于某些固定的 N，如果数组 A 是整数 1, 2, ..., N 组成的排列，使得：对于每个 i < j，都不存在 k 满足 i < k < j 使得 A[k] * 2 = A[i] + A[j]。则A称为A是漂亮数组。例如：N=4，那么数组为[2,1,4,3]

### 分析
- 分治法步骤，先分解问题，在分解问题之前我们要想下，如果arr1满足条件，arr2也满足条件，怎么合并才能让[arr1,arr2]满足条件呢？方法就是arr1是里全部是奇数，arr2里全部是偶数，这样一定能保证[arr1,arr2]符合条件(想不通可以参看15高频算法问题4).
- 还有一个问题是如何让N个数字全部排列起来呢？思考，如果一个数组是arr,那么k*arr+b是否也是满足漂亮数组的条件呢？（是的）从这个角度我们直到如果要得到一个2N长度的漂亮数组，可以先得到长度为N的漂亮数组arr，然后使得arr1=2\*arr,arr2=2\*arr-1,这样[arr1,arr2]是否就是2N的漂亮数组了呢(是的)
- 经过以上分析我们的递归函数可以是返回一个长度为N/2的漂亮数组，再通过[2\*arr, 2\*arr-1]的方法得到长度为N的漂亮数组,这样的方式对N为偶数时成立，当N为奇数时会导致损失数字，所以我们需要递归两次，分别得到奇数变换需要的数组(奇变换表示*2-1)，和偶变换需要的数组
### 优化
- 增加一个字典保存不同N的结果，在计算odd_ans和even_ans之前判断一下

In [12]:
def process(N:int):
    if N==1:
        return [1]
    odd_ans = process((N+1)//2) # 奇变换数列
    even_ans = process(N//2)    # 偶变换数列
    ans = [i*2 for i in even_ans] + [i*2-1 for i in odd_ans]
    return ans
process(5)

[4, 2, 3, 5, 1]

## 问题3：[戳气球](https://leetcode-cn.com/problems/burst-balloons/)
### 题目
给定一个n个气球，每个气球上都标有一个数字，这些数字存在数组 nums 中。我们可以随意戳第i个气球，如果戳的是的第i个气球，那么将会获得nums[i - 1] * nums[i] * nums[i + 1]枚硬币。戳破之后，nums数量会减一，继续戳，最后直到没有气球，如果第i号气球两边超出边界则*1，例如[4,2]，我戳破第0位置的气球则会得到1\*4\*2枚银币。问获得的银币数量最多为多少。

### 分析
- 三步走：
    - 分解问题: 如当前选择第i个位置作为刺破，则需要得到subarr=arr[:i] + arr[i+1:]最大的结果
    - 递归函数：输入为arr，和当前硬币钱数，当输入的arr是一个[]时, 则可以返回当前得到的累计值结果
    - 合并: 计算最大硬币数量
- 以上结果虽然能解决，但是复杂度过高，因为每次subarr仅仅是arr少了一个字符。

### 优化
- 我们可以再arr前和末尾增加1，这样就不用处理边界了。
- 由于正向思考，来模拟刺破情景会导致复杂度过高，其原因是当刺破第i个气球的时候会改变第i-1和第i+1气球的相邻关系，导致我们每次计算的sub_arr还是很大。我们反向思考如果第i个气球是最后区间(left，right)中最后一个被刺破的，意味着最后其实剩余的只有(left, , ,i, , right)，此时我们那么区间(i, right)最大银币数，和(left, i)区间的最大银币数，是相互隔离的，所以(left, right)中最后刺破i位置能获得的最大硬币数应该是arr[left] * arr[i] * arr[right]+process(left, i)+process(i, right)

In [1]:
### 优化前+记忆搜索的结果
cash = dict()
def process(arr):
    if len(arr) == 1:
        return arr[0]
    max_sub_count = 0
    for i in range(len(arr)):
        if i == 0:
            value = 1 * arr[i] * arr[i+1]
        elif i == len(arr)-1:
            value = arr[i-1] * arr[i] * 1
        else:
            value = arr[i-1] * arr[i] * arr[i+1]
        sub_arr = arr[:i] + arr[i+1:] # 计算子数组最大count
        # 记忆机制
        if str(sub_arr) in cash:
            sub_count = cash[str(sub_arr)]
        else:
            sub_count = process(sub_arr)

        max_sub_count = max(max_sub_count, sub_count+value)
    cash[str(arr)] = max_sub_count
    return max_sub_count
nums = [3,1,5,8]
process(nums)

167

In [3]:
# 采用@lru_cache()来替代全局字典，作为函数结果记录=记忆搜索。优化后
import functools

@functools.lru_cache()
def process(left, right):
    if left+1 == right:
        return 0
    
    max_sub_count = 0
    for i in range(left+1, right): # 刺破的气球不包括left和right位置
        value = nums[left] * nums[i] * nums[right] # 这里注意是arr[left]和arr[right]，此处意义是最后刺破第i个气球
        left_sub = process(left, i)
        right_sub = process(i, right)
        max_sub_count = max(max_sub_count, value+left_sub+right_sub)
    return max_sub_count

nums = [3,1,5,8]
nums = [1] + nums + [1]
process(0, len(nums)-1)


167

1